# Jenkins building a basic end-end CI/CD pipeline of a maven project.



1.   Jenkins is could be spinned up on a Virtual Machine or directy install it on pc or mac.
2.   Installing SonarQube you can set it up directly on host or a docker container.

**Plugins required:** jdk, SonarQube, OWASP, Docker Related



In [ ]:
docker run -d -p 9000:9000 sonarqube:lts-community

In [ ]:
# Check if the sonarqube conatiner is up and running
docker ps


It is crucial when you spin up the below script it should have sonarqube running or the build will fail.


Write a build pipeline
take a base hello world template
1. Git chekcout
2. Then configuring jd, sonarscanner, maven.
3. Basically whatever plugin we install we have to configure it at Dashboard > Manage Jenkins > Global Tool Configuration
4. So going ahead configure plugins like OWASP, Docker etc.



**The question here arises do we have to remember the script which is used for checkout and lot of other places answer is no:**
1. Click on pipeline syantax on configure window or Go to Dashboard > CI_Pipeline > Pipeline Syntax
2. Navigate in page to generate pipeline script
3. Select a sample step like in this case git and generate the script.
4. for docker build image and push we are generatin withDockerregistry Sample Step.

In [ ]:
pipeline {
    agent any

    tools {
        jdk 'jdk11'
        maven 'maven3'

    }

    environment {
        SCANNER_HOME= tool 'sonar-scanner'
    }

    stages {
        stage('Git Checkout') {
            steps {
               git branch: 'main', url: 'https://github.com/jaiswaladi246/Ekart.git'
            }
        }

        stage('Compile Source code') {
            steps {
               sh "mvn clean compile"

            }
        }

        stage('Sonarqube Analysis') {
            steps {
            sh '''
            $SCANNER_HOME/bin/sonar-scanner \
            -Dsonar.host.url=http://172.105.17:9000/ \
            -Dsonar.login=squ_9e347cf4e7aacbd54cf6902bac23449b2b49bced \
            -Dsonar.projectName=ekart \
            -Dsonar.java.binaries=. \
            -Dsonar.projectKey=ekart
        '''

            }
        }

        stage('OWASP') {
            steps {
               dependencyCheck additionalArguments: ' --scan ./', odcInstallation: 'DP'
               dependencyCheckPublisher pattern: '**/dpendency-check-report.xml'
            }
        }

        stage('Build App') {
            steps {
               sh "mvn clean install -DskipTests"
            }
        }

          stage('This is to trigger the CD') {
            steps {
               build job: "name of the Cd job", wait: true

            }
        }
    }
}


Now create a separate pipeline which will be your Continious Deployment Pipeline


In [ ]:
pipeline {
    agent any

    stages {
        stage('Docker Deploy to container') {
            steps {
                withDockerRegistry (credentialsId: 'credential will be geenrated by pipeline syntax when we will add auth creds', toolName: 'docker') {
                    sh "docker run -d --name <container name> -p 8070:8070 <image name>"
                }
            }
        }
    }
}


If any issue arises regarding credentials, those can always be updated under manage jenkins > credentials.

# Great your first Maven CI/CD is ready!